# Advent of Code 2018
http://adventofcode.com/2018

## Day 1
Puzzle input: `input01.txt`
### Part 1

In [1]:
with open('input01.txt') as f:
    freq_change = [int(n) for n in f]
    
print(sum(freq_change))

576


### Part 2

In [2]:
last_freq = 0
freq_history = set()
n = len(freq_change)

i = 0
while last_freq not in freq_history:
    freq_history.add(last_freq)
    last_freq += freq_change[i]    
    i = (i+1) % n
    
print(last_freq)

77674


## Day 2
Puzzle input: `input02.txt`
### Part 1

In [3]:
with open('input02.txt') as f:
    words = [w.strip() for w in f]

In [4]:
from collections import Counter

twos, threes = 0, 0
for w in words:
    c = Counter(w)
    if 2 in c.values(): twos += 1
    if 3 in c.values(): threes += 1
        
print(twos * threes)

5904


### Part 2

In [5]:
from itertools import combinations

def diff_count(w1, w2):
    return sum(a != b for a,b in zip(w1, w2))

def common(w1, w2):
    return ''.join(a for a,b in zip(w1, w2) if a == b)

for w1, w2 in combinations(words, 2):
    if diff_count(w1, w2) == 1: print(common(w1, w2))

jiwamotgsfrudclzbyzkhlrvp
